### Введение
Изначально появилась такая задача: локализовать точку на плоскости. В практическом применении это выглядит примерно так: мы получаем каким-то образом свои координаты, и по ним требуется определить местоположение на карте. Так как карта заранее известна, ее можно предподсчитать, чтобы потом время обработки запроса было как можно меньше. Но карты обычно довольно большие, так что предподсчет должен использовать как можно меньше дополнительной памяти. Трапецоидная карта в какой-то мере позволяет решить эту задачу, используя в среднем $O(n)$ памяти, $O(n\log n)$ времени на предподсчет и $O(\log n)$ времени на запрос.

### Трапецоидная карта
Изначально имеется множество непересекающихся отрезков, лежащих на плоскости. Это множество "условно" помещается в bounding box $R$ – прямоугольник, ограничивающий все отрезки. Также условимся, что среди всех вершин отрезков никакие две различные вершины не лежат на одной вертикальной прямой (но при этом они могут совпадать). В дальнейшем эти ограничения можно будет снять.

Трапецоидная карта – структура данных для локализации точки среди этих отрезков. Она получается следующим путем: из каждой вершины выпускается два вертикальных луча, вверх и вниз, до первого пересечения с другим отрезком или с $R$.

![Пример трапецоидной карты](images/tmap_example.jpg)
<i><center>Рисунок №1. Пример трапецоидной карты</center></i>

Рассмотрим, что из себя представляет отдельно каждый трапецоид:

<b>Лемма.</b> <i>Любой трапецоид ограничен одним или двумя вертикальными и двумя не вертикальными отрезками</i>
- Обозначим наш трапецоид $f$. Для начала покажем, что $f$ выпуклый. Любая угловая точка у $f$ по построению является либо концом отрезка, либо пересечением вертикального луча с другим отрезком или с $R$, либо это один из углов $R$. Так как лучи вертикальные, то угол пересечения с отрезком не превзойдет 180 градусов, а угол с участием $R$ равен 90 градусам. Следовательно, $f$ выпуклый, так как вертикальные лучи устраняют все невыпуклости.
- Из построения $f$ следует, что должно быть не менее 2 не вертикальных отрезков. Допустим, их будет больше 2. Тогда как минимум 2 из них будут смежными с верхней (или нижней) стороны, и у них будет общая точка. Но в ней должен был быть вертикальный луч, который создал бы дополнительный трапецоид! Значит, не вертикальных отрезков ровно два. Также по построению $f$ не может иметь больше 2 вертикальных отрезков, но один из них может отсутствовать (верхний и нижний отрезок имеют одну общую точку, а трапецоид будет треугольником).
   
Отсюда и берется название трапецоидных карт, так как трапецоид представляет из себя либо трапецию, либо треугольник.

Обозначим отрезок, лежащий сверху трапецоида, $\mathit{top}(\Delta)$, и аналогично лежащий снизу $\mathit{bottom}(\Delta)$. Рассмотрим возможные варианты расположения левого отрезка:
- он отсутствует, вместо него точка пересечения $\mathit{top}(\Delta)$ и $\mathit{bottom}(\Delta)$;
- он образован лучом, идущим вниз из левой точки $\mathit{top}(\Delta)$;
- он образован лучом, идущим вверх из левой точки $\mathit{bottom}(\Delta)$;
- он образован двумя лучами из правой точки отрезка, лежащего слева от $\Delta$;
- это левая граница $R$.

![Случаи расположения leftp](images/leftp_cases.jpg)
<i><center>Рисунок №2. Варианты расположения левой точки трапецоида</center></i>

В каждом случае (за исключением пятого) левый отрезок определяла одна точка $p$ – вершина одного из отрезков. Обозначим ее как $\mathit{leftp}(\Delta)$ (в случае для $R$ это будет $None$, так как координата его левой нижней точки не известна, да и сам $R$ нереален). Аналогичным способом получим 5 возможных случаев для расположения правого отрезка, обозначив правую вершину как $\mathit{rightp}(\Delta)$. Заметим, что трапецоид однозначно задается набором $\mathit{top}(\Delta)$, $\mathit{bottom}(\Delta)$, $\mathit{leftp}(\Delta)$, $\mathit{rightp}(\Delta)$.

А сколько же всего трапецоидов мы получим?

<b>Лемма.</b> <i>Трапецоидная карта, построенная на $n$ отрезках, содержит не более $6n+4$ вершин и $3n+1$ трапецоидов</i>
- Вершиной трапецоида может являться либо "условная" вершина $R$ (всего их четыре), либо конец отрезка ($2n$ вершин), либо конец вертикального луча, выходящего из конца отрезка ($2n \cdot 2=4n$ вершин) – итого не более $6n+4$ вершин.
- Для ограничения числа трапецоидов рассмотрим точку $\mathit{leftp}(\Delta)$. Она является либо концом отрезка, либо $\mathit{None}$. Если вершина $\mathit{None}$, то этот трапецоид ограничен левой стороной $\mathit{R}$, а такой трапецоид будет один. Правый конец каждого отрезка задает не более одной $\mathit{leftp}(\Delta)$ (4 случай на рис.№2); левый конец отрезка – не более двух $\mathit{leftp}(\Delta)$ (2-3 случаи на рис.№2). Однако в 1 случае, когда $k>1$ отрезков имеют общую левую точку, $\mathit{leftp}(\Delta)$ может быть общей сразу для $k+1$ трапецоидов. В этом случае условимся, что каждый из $k$ отрезков задает $\mathit{leftp}(\Delta)$ лишь для верхнего и нижнего трапецоида, тогда каждому трапецоиду соответствует два отрезка и все хорошо. Значит, каждая точка $\mathit{leftp}(\Delta)$ задает не более трех трапецоидов, а общее количество трапецоидов не превышает $3n+1$.

В дальнейшем нам понадобятся трапецоиды с общей вертикальной прямой – назовем их соседями. По нашему условию на одной вертикальной прямой лежит максимум одна вершина, значит трапецоид может иметь до двух соседних трапецоидов слева и справа (иначе бы могло быть любое число соседей). На рис.№2 в первом и пятом случаях у трапецоида левые соседи отсутствую, во втором присутствует только нижний, в третьем только верхний, а в четвертом сразу оба соседа. Также трапецоиду необходимо знать узлы, которые на него указывают (в процессе построения локализационной структуры станет ясно, зачем), а в итоге структура для трапецоида будет выглядеть примерно так:

In [ ]:
"""Структура, описывающая отрезок"""
class SegmentExample():
    "Предполагаем, что p и q упорядочены лексикографически"
    def __init__(self, p, q):
        "Левая точка"
        self.p = p
        "Правая точка"
        self.q = q

"""Структура, описывающая трапецоид"""
class TrapezoidExample():

    def __init__(self, top, bottom, leftp, rightp):
        "Верхний и нижний отрезки"
        self.top = top
        self.bottom = bottom
        
        "Левая и правая точки"
        self.leftp = leftp
        self.rightp = rightp
        
        "Соседи трапецоида"
        self.leftnb = [None, None]
        self.rightnb = [None, None]
        
        "Ссылки на узлы локализационной структуры"
        self.links = []

### Локализация точки на трапецоидной карте
Рассмотрим процесс локализации точки $q$ на трапецоидной карте $J$, изображенной на рис.№3.

![Пример локализации в J](images/tmap_localization_example.jpg)
<i><center>Рисунок №3. Пример построенной трапецоидной карты</center></i>

Для быстрой локализации точки заведем локализационную структуру $D$. Она представляет из себя ациклический ориентированный граф с одним корнем (практически дерево, но в узел графа может входить более одного ребра), в котором листами являются трапецоиды. <b>Важное замечание:</b> каждому трапецоиду в графе соответствует ровно один лист!

Все узлы графа делятся на 2 типа

- $X$ соответствует вершине отрезка ($p_i$ левая вершина, а $q_i$ правая)
- $Y$ соответствует самому отрезку $s_i$

У каждого узла графа 2 исходящих ребра. При запросе локализации точки $q$ на трапецоидной карте $J$ мы спускаемся по графу от корня к нужному трапецоиду. В случае узла $X$ мы сравниваем вершины лексикографически: если $q$ меньше, то идем по синему ребру, иначе по оранжевому. В случае узла типа $Y$ мы проверяем, лежит ли $q$ выше или ниже отрезка. Если $q$ выше, то идем по синему ребру, иначе по оранжевому. В конце мы доходим до соответствующего точке $q$ трапецоида.

![Пример локализации в D](images/localization.gif)
<i><center>Рисунок №4. Пример локализации точки в поисковой структуре</center></i>

Также могут быть случаи, когда $q$ совпадает с вершиной или принадлежит отрезку. С одной стороны, нельзя не признать, что $q$ все ещё принадлежит трапецоиду (причем, сразу нескольким), и при выборе любого ребра, исходящего из текущего узла, мы придем к нужному трапецоиду. С другой стороны, нельзя не согласиться, что вершина или отрезок более точно локализуют точку запроса, нежели трапецоид, значит мы можем выдать в качестве ответа отрезок, на котором лежит $q$ (или в случае вершины отрезок, который ей соответствует).

### Построение поисковой структуры и трапецоидной карты
Изначально карта $J_0$ состоит из единственного трапецоида, у которого отсутствуют и соседи, и верхний/нижний отрезок, и левая/правая точка (фактически, это весь $R$), и к тому же он лежит в корне $D$. Алгоритм построения трапецоидной карты инкрементальный: в уже существующую карту по одному добавляются новые отрезки. Что необходимо сделать при добавлении очередного отрезка $s_i$:
- найти трапецоиды $\Delta_0,\Delta_1,\ldots,\Delta_k$, которые пересекает $s_i$;
- удалить их из $J_{i-1}$ и заменить на новые трапецоиды, появившиеся при вставке $s_i$
- заменяем листы из $D_{i-1}$, соответствующие старым трапецоидам, на новые

Поиск $\Delta_0,\Delta_1,\ldots,\Delta_k$ выполняется довольно просто. Сначала мы находим $\Delta_0$, локализуя левую точку отрезка $s_i$ в $D_{i-1}$ за $O(h)$, где $h$ – высота $D_{i-1}$. Далее легко получить $\Delta_1,\ldots,\Delta_k$, проходя вправо по соседям. Для проверки, верхним или нижним будет следующий трапецоид, нужно проверить поворот точки $\mathit{rightp}(\Delta_j)$ относительно прямой $s_i$. Поиск остановится, когда правая вершина $s_i$ либо окажется левее $\mathit{rightp}(\Delta_k)$, либо попадет в крайний правый трапецоид, либо конец отрезка совпадет с $\mathit{rightp}(\Delta_j)$. Приведём код для обхода соседних трапецоидов:

In [ ]:
import numpy as np

"Возвращает список трапецоидов, которые пересекает отрезок"
def intersectSegment(s, leftTr):
    answer = [leftTr]
    "Поиск остановится, если у трапецоида отсутствует правый вертикальный отрезок или нету соседей."
    while leftTr.rightp != None and leftTr.rightnb != [None, None]:
        if s.q[0] < leftTr.rightp[0]:
            "Нашли крайний правый трапецоид"
            break
        "Иначе считаем поворот точки rightp относительно прямой s"
        sign = np.sign(np.linalg.det(np.array([s.p, s.q]) - leftTr.rightp))
        if sign == 1:
            "Отрезок пересекает нижнего соседа"
            leftTr = leftTr.rightnb[1]
        elif sign == -1:
            "Отрезок пересекает верхнего соседа"
            leftTr = leftTr.rightnb[0]
        else: #sign == 0
            "Отрезок попал ровно в вершину, поиск останавливается"
            assert s.q == leftTr.rightp
            break
        answer.append(leftTr)
    return answer

Таким образом, мы получим $\Delta_0,\Delta_1,\ldots,\Delta_k$ за $O(h+k)$.

Далее необходимо удалить старые трапецоиды и на их место вставить новые. Сначала разберем простой случай, когда $s_i$ целиком попал в один трапецоид. $s_i$ вместо $\Delta$ порождает 4 новых трапецоида. Необходимо обновить указатели у соседей $\Delta$ и новых трапецоидов, а также заменить в $D_{i-1}$ лист, соответствовавший $\Delta$, на новое поддерево высоты 3, как показано на рис.№5. Этот случай обрабатывается за $O(1)$.

![Один трапецоид](images/single_s_i.jpg)
<i><center>Рисунок №5. Вставка отрезка в трапецоид</center></i>

Теперь рассмотрим сложный случай, когда $s_i$ пересекает $\Delta_0,\Delta_1,\ldots,\Delta_k$. Необходимо по-разному обработать 3 случая
- левый конец отрезка внутри $\Delta_0$
- правый конец отрезка внутри $\Delta_k$
- отрезок полностью пересекает трапецоид

В первых двух случаях концы $s_i$ порождают новые вертикальные лучи, то есть необходимо разбить $\Delta_0$ и/или $\Delta_k$ на три трапецоида. Кроме того, $s_i$ пересечет некоторые другие вертикальные лучи, значит надо подразбить трапецоиды вдоль $s_i$, начиная с $\Delta_0$. При переходе от $\Delta_i$ к $\Delta_{i+1}$ смотрим, с какой стороны от $s_i$ лежит точка $\mathit{rightp}(\Delta_i)$. Если она лежит сверху, то трапецоид ниже $s_i$ продолжится вдоль отрезка, а трапецоид сверху $s_i$ закончится. Выставляем корректно все вершины и всех соседей для трапецоидов, за исключением того, что у нижнего трапецоида $\mathit{rightp}(\Delta_{low})=None$. Запомним его для правильной расстановки указателей на следующем шаге. Мы сможем выставить ему $\mathit{rightp}$, когда появится такой трапецоид $\Delta_j$, что $\mathit{rightp}(\Delta_j)$ будет ниже $s_i$, или мы дойдем до конца отрезка $s_i$. С трапецоидами, тянущимися сверху, поступаем аналогично.

В $D_{i}$ листы, соответствовавшие $\Delta_0,\Delta_1,\ldots,\Delta_k$, заменяются на новые поддеревья. На 6 рисунке первый случай отсутствует, так как $\mathit{leftp}(\Delta_0)$ совпадает с левой вершиной $s_i$. Второй случай, наоборт, присутствует: лист $\Delta_3$ заменяется на поддерево высотой 3 с узлом типа $X$ и $Y$, которые указывают на 3 новых трапецоида. Все остальные трапецоиды подпадают под третий случай, где лист заменится на поддерево высоты 2 с узлом типа $Y$, указывающим на два новых трапецоида. Все это делается за $O(k)$, так как для каждого из $k+1$ трапецоида выполняется $O(1)$ действий. В итоге высота дерева увеличивается не более, чем на 2, что видно на рис.№6:

![Несколько трапецоидов](images/multi_s_i.jpg)
<i><center>Рисунок №6. Вставка отрезка в несколько трапецоидов</center></i>

Теперь вернемся к нашей трапецоидной карте (см. рис.№3) и рассмотрим пример вставки очередного отрезка.

![Вставка в J](images/map_insert.gif)
<i><center>Рисунок №7. Обновление карты после вставки отрезка</center></i>

Как видно, новый отрезок $s_{17}$ пересекает 4 трапецоида. Вершины отрезка попадают внутрь трапецоидов $\Delta_0$ и $\Delta_{14}$, поэтому в новом дереве вместо этих трапецоидов сначала вставляются $p_{17}$ и $q_{17}$ (выделены желтым), а уже затем сам отрезок (как и для двух других трапецоидов $\Delta_{5}$ и $\Delta_{10}$).

![Вставка в D](images/tree_insert.gif)
<i><center>Рисунок №8. Обновление поисковой структуры после вставки отрезка</center></i>

В сумме локализация и вставка нового отрезка $s_i$ займут $O(h+k)$ времени. Таким образом мы получим корректную трапецоидную карту $J$ и поисковую структуру $D$, так как на каждом шаге добавление нового отрезка было корректным.

### Асимптотика и память
Порядок добавления отрезков очень важен, так как при добавлении нового отрезка высота $D$ может увеличится до 3, а в худшем случае высота дерева может составить $3n$. Несложно придумать последовательность отрезков, демонстрирующую этот случай. Тогда алгоритм будет строить дерево за $O(n^2)$, а локализация точки будет выполняться за $O(n)$. Для сглаживания этой неприятности отрезки добавляют в случайном порядке, что дает более приемлемую временную оценку.

Зафиксируем множество $S$ из $n$ отрезков и точку запроса $q$. Всего возможно $n!$ перестановок отрезков, а значит $n!$ различных структур $D$. В этом случае мы можем оценить ожидаемое значение высоты $D$. Добавим немножко теорвера: обозначим количество узлов на пути локализации $q$, созданных на $i$-ой итерации алгоритма, за $x_i$ – это случайная величина. Найдем матожидание длины пути:

$E\left[\sum_{i=1}^{n}{x_i}\right]=\sum_{i=1}^n{E\left[x_i\right]}$

Также мы знаем, что $x_i\leq3$. Обозначим $p_i$ как вероятность встретить на пути локализации $q$ узел, созданный на $i$-ой итерации. Ясно, что $E\left[x_i\right]\leq3p_i$. Оценим $p_i$. Необходимо понять, когда $p_i\neq0$. Это так только в том случае, когда $q \in \Delta_{i-1}$, но на шаге $i$ трапецоид $\Delta_{i-1}$ был удален, а точка $q$ перешла в трапецоид $\Delta_i$. Применим так называемый "backwards-analysis": на $i$ шаге удалим случайный отрезок $s_k \colon k \in [1,i]$ и оценим вероятность исчезновения трапецоида $\Delta_i$. Это произойдет в 4 случаях:
- $\mathit{top}(\Delta_i)=s'_i$
- $\mathit{bottom}(\Delta_i)=s'_i$
- $\mathit{leftp}(\Delta_i)$ – конец отрезка $s'_i$
- $\mathit{rightp}(\Delta_i)$ – конец отрезка $s'_i$

Так как $s_i$ вставлялись в случайном порядке, то для каждого случая вероятность того, что $s'_i=s_i$ равна $\dfrac{1}{i}$, а в сумме она не превосходит $\dfrac{4}{i}$. Таким образом $\sum_{i=1}^n{E\left[x_i\right]} \leq \sum_{i=1}^n{3p_i} \leq \sum_{i=1}^n{\dfrac{12}{i}} = 12\sum_{i=1}^n{\dfrac{1}{i}} = 12H_n$, где $H_n$ – гармонический ряд, который асимптотически равен $12ln(n) \approx O(\log n )$. Значит, ожидаемое время локализации составит $O(\log n )$.

Теперь вернемся к размеру $D$. В худшем случае на каждой итерации алгоритма новый отрезок будет пересекать все трапецоиды, и тогда размер структуры составит $O(n^2)$. Найдем ожидаемый объем памяти. Мы знаем, что в конце алгоритма у нас будет $O(n)$ листов в $D$, тогда размер $D$ составит $O(n) + \sum_{i=1}^{n}{E\left[x_i-1\right]} = O(n) + \sum_{i=1}^{n}{E\left[x_i\right]}$, где $x_i$ – количество трапецоидов, созданных на $i$-ой итерации. Надо ограничить $E\left[x_i\right]$.

Снова применим backwards-analysis. Зафиксируем набор отрезков $S_i$. Введем новую функцию:

$\delta(\Delta, s) = \begin{cases}
1, &\text{если }\Delta\text{ исчезнет при удалении }s_i\\
0, &\text{иначе}
\end{cases}$

$\Delta$ может исчезнуть при удалении $\mathit{top}(\Delta)$, $\mathit{bottom}(\Delta)$, точки $\mathit{leftp}(\Delta)$ или $\mathit{rightp}(\Delta)$ (если они присутствуют у $\Delta$). Значит, на $\Delta$ влияет не более 4 отрезков, тогда $\sum_{s \in S_i}{\sum_{\Delta \in J_i} {\delta(\Delta, s)}} \leq 4 \left|J_i\right| = O(i)$. С учетом того, что вероятность удаления отрезка равна $\dfrac{1}{i}$, найдем матожидание: $E\left[x_i\right] = \dfrac{1}{i}\sum_{s \in S_i}{\sum_{\Delta \in J_i}{\delta(\Delta, s)}} \leq \dfrac{O(i)}{i} = O(1)$. Таким образом, за одну итерацию объем памяти увеличится на $O(1)$, а всего структура займет $O(n)$ памяти.

Остается только получить время работы алгоритма, что довольно просто. Уже известно, что добавление отрезка занимает $O(h+k)$, но ожидаемое значение составляет $O(\log n + 1) \approx O(\log n)$, а для $n$ отрезков потребуется $O(n\log n)$ времени.

### Вырожденные случаи

Ранее мы условились, что среди всех вершин отрезков любые две вершины не лежат на одной вертикальной прямой (но при этом они могут совпадать). Разрешим эти случаи. Для этого надо "слегка" повернуть систему координат. При достаточно малом угле поворота никакие две точки не будут лежать на одной вертикальной прямой, и все будет хорошо. Фактически, достаточно упорядочить точки лексикографически: будем считать, что точка $p(x,y_p)$ лежит правее точки $q(x,y_q)$, если $y_q<y_p$, и наоборот.

Более формально, нам необходимо выполнить следующее преобразование для всех точек отрезков:

$\varphi : \left( \begin{array}{c} x\\ y\end{array}\right) 
\rightarrow
\left( \begin{array}{c} x + \varepsilon y\\ y\end{array}\right)$

Так мы отобразим вертикальную прямую на прямую с углом наклона $1/\varepsilon $. Если $\varepsilon$ достаточно мал, то это преобразование не поменяет исходный порядок точек по координате $x$.  Конечно, появятся новые вырожденные трапецоиды, которые не могли возникнуть в исходной система координат. Но так как количество отрезков не изменилось, время работы алгоритма останется тем же.

На практике достаточно упорядочить точки лексикографически, и тогда мы сможем добавлять как вертикальные отрезки, так и отрезки с вершинами, лежащими на одной вертикальной прямой.

## Пример реализации алгоритма
На данный момент алгоритм корретно вставляет отрезки только целиком в существующий трапецоид! Предлагается улучшить алгоритм и дописать вставку отрезка с учетом возможного пересечения нескольких трапецоидов.

In [ ]:
%matplotlib

from solution import *

import matplotlib.pyplot as plt
import matplotlib.lines as lines

tmap = TrapezoidalMap()
nextPoint = None

def OnClick(event):
    global nextPoint, tmap
    if not event.dblclick:
        plt.plot(event.xdata,event.ydata,'ro')
        if nextPoint == None:
            nextPoint = [event.xdata, event.ydata]
            plt.pause(0.05)
        else:
            tempPoint = [event.xdata, event.ydata]
            if (nextPoint[0] > tempPoint[0]):
                nextPoint, tempPoint = tempPoint, nextPoint
            seg = Segment(nextPoint, tempPoint)
            tmap.insert(seg)
            "Дорисуем последние 3 трапецоида"
            for i in range(3):
                data = points(tmap.tr[len(tmap.tr) - 1 - i])
                plt.plot(data[0], data[1], 'g')
                plt.pause(0.05)
            nextPoint = None

plt.figure(1)
fig = plt.gcf()
cid_up = fig.canvas.mpl_connect('button_press_event', OnClick)
plt.plot([0, 0, 5, 5], [0, 5, 5, 0], 'g')
plt.show()